## Preprocessing

In [41]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [42]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_dropped = application_df.drop(['EIN','NAME'], axis=1)

In [43]:
# Determine the number of unique values in each column.
application_df_dropped.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [44]:
# Look at APPLICATION_TYPE value counts for binning
application_df_dropped.value_counts('APPLICATION_TYPE')

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64

In [45]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cut_off = 100 

application_counts = application_df_dropped.value_counts('APPLICATION_TYPE')
application_types_to_replace = application_counts[application_counts < cut_off].index.to_list()
# Replace in dataframe
for app in application_types_to_replace:
    application_df_dropped['APPLICATION_TYPE'] = application_df_dropped['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_dropped.value_counts('APPLICATION_TYPE')

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
dtype: int64

In [46]:
# Look at CLASSIFICATION value counts for binning
application_df_dropped.value_counts('CLASSIFICATION')

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [47]:
# You may find it helpful to look at CLASSIFICATION value counts >1
 
application_df_dropped.value_counts('CLASSIFICATION')[application_df_dropped.value_counts('CLASSIFICATION') > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1267        2
C1246        2
C1256        2
C3200        2
C1234        2
dtype: int64

In [48]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1 
classification_counts = application_df_dropped.value_counts('CLASSIFICATION')
classifications_to_replace = classification_counts[classification_counts <= cutoff].index.to_list()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df_dropped.loc[application_df_dropped['CLASSIFICATION'] == cls, 'CLASSIFICATION'] = "Other"
    # application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
# Check to make sure binning was successful
new_classification_counts = application_df_dropped.value_counts('CLASSIFICATION')
new_classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Other       26
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C3200        2
C1267        2
C1256        2
C1246        2
C1234        2
dtype: int64

In [49]:
application_df_dropped.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [50]:
# Convert categorical data to numeric with `pd.get_dummies`

categorical_cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
dummies = pd.get_dummies(application_df_dropped[categorical_cols])
application_df_dropped = application_df_dropped.drop(categorical_cols, axis=1)
application_df_dropped = pd.concat([application_df_dropped, dummies], axis=1)
application_df_dropped.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [51]:
# Split our preprocessed data into our features and target arrays
target = application_df_dropped['IS_SUCCESSFUL']
features = application_df_dropped.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [52]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [58]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_features = 84
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=40, activation="relu", input_dim=input_features))
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=20, activation='relu'))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) 
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 40)                3400      
                                                                 
 dense_4 (Dense)             (None, 20)                820       
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 4,241
Trainable params: 4,241
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [61]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100
429/429 [==============================] - 1s 955us/step - loss: 20053.4004 - accuracy: 0.4831
Epoch 2/100
429/429 [==============================] - 0s 955us/step - loss: 15914.8105 - accuracy: 0.4984
Epoch 3/100
429/429 [==============================] - 0s 921us/step - loss: 63186.0508 - accuracy: 0.4974
Epoch 4/100
429/429 [==============================] - 0s 979us/step - loss: 41384.1133 - accuracy: 0.4946
Epoch 5/100
429/429 [==============================] - 0s 985us/step - loss: 32459.6543 - accuracy: 0.5107
Epoch 6/100
429/429 [==============================] - 0s 1ms/step - loss: 22895.2832 - accuracy: 0.4961
Epoch 7/100
429/429 [==============================] - 0s 1ms/step - loss: 22980.4277 - accuracy: 0.5076
Epoch 8/100
429/429 [==============================] - 0s 974us/step - loss: 8411.6953 - accuracy: 0.5044
Epoch 9/100
429/429 [==============================] - 0s 1ms/step - loss: 27027.6641 - accuracy: 0.4924
Epoch 10/100
429/429 [======================

In [62]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.6778 - accuracy: 0.6328 - 246ms/epoch - 1ms/step
Loss: 0.6778001189231873, Accuracy: 0.6327988505363464


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE